In [1]:
'''
https://www.kaggle.com/avijitghosh82/category-wise-analysis-usa
https://www.kaggle.com/ievgenvp/youtube-trends-eda/notebook
https://www.kaggle.com/avijitghosh82/category-wise-analysis-usa
https://www.kaggle.com/duanerich/eda-on-youtube-s-data
https://www.kaggle.com/ajayshelar/readability-analysis-of-youtube-titles
https://www.kaggle.com/ambllamas/exploring-trending-youtube-videos
'''

'\nhttps://www.kaggle.com/avijitghosh82/category-wise-analysis-usa\nhttps://www.kaggle.com/ievgenvp/youtube-trends-eda/notebook\nhttps://www.kaggle.com/avijitghosh82/category-wise-analysis-usa\nhttps://www.kaggle.com/duanerich/eda-on-youtube-s-data\nhttps://www.kaggle.com/ajayshelar/readability-analysis-of-youtube-titles\nhttps://www.kaggle.com/ambllamas/exploring-trending-youtube-videos\n'

In [2]:
'''
Técnicas Envolvidas
• Análise descritiva de dados.
• Classificação binária.
• Clustering.
Desafios
Para esse projeto, alguns desafios são:
• Classificação conforme like/dislike vinculado aos comentários, extrapolar like/dislike de quem comentou e não
classificou.
• Clustering baseado em comentários.
• Análise de mudança de sentimento no decorrer do tempo.
'''

'\nTécnicas Envolvidas\n• Análise descritiva de dados.\n• Classificação binária.\n• Clustering.\nDesafios\nPara esse projeto, alguns desafios são:\n• Classificação conforme like/dislike vinculado aos comentários, extrapolar like/dislike de quem comentou e não\nclassificou.\n• Clustering baseado em comentários.\n• Análise de mudança de sentimento no decorrer do tempo.\n'

## Imports

In [3]:
import pandas as pd
import numpy as np
from collections import Counter
import operator
import json
import matplotlib.pyplot as plt
from keras.preprocessing.sequence import pad_sequences
%matplotlib inline

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Carregando os dados

### Categorias

In [4]:
f = open("data/US_category_id.json")
data = f.read()
f.close()
out = json.loads(data)
categories = pd.DataFrame([
    {'category_id':x['id'],
     'category':x['snippet']['title'],
     'assignable':x['snippet']['assignable']
    } for x in out['items']
])

In [5]:
categories.head()

,assignable,category,category_id
0,True,Film & Animation,1
1,True,Autos & Vehicles,2
2,True,Music,10
3,True,Pets & Animals,15
4,True,Sports,17


In [6]:
categories['category_id'] = categories['category_id'].astype(int)

### Vídeos e comentários (EN)

In [55]:
#USvideos = pd.read_csv('data/USvideos.csv',error_bad_lines=False)
UScomments = pd.read_csv('data/UScomments.csv',error_bad_lines=False)

b'Skipping line 41589: expected 4 fields, saw 11\nSkipping line 51628: expected 4 fields, saw 7\nSkipping line 114465: expected 4 fields, saw 5\n'
b'Skipping line 142496: expected 4 fields, saw 8\nSkipping line 189732: expected 4 fields, saw 6\nSkipping line 245218: expected 4 fields, saw 7\n'
b'Skipping line 388430: expected 4 fields, saw 5\n'
/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Manipulando os dados

### Realizando *merge* das categorias com os vídeos

In [49]:
USvideos['category_id'] = USvideos['category_id'].astype(int)
USvideos['date'] = USvideos['date'].apply(lambda x: pd.to_datetime("{:05.2f}".format(x).replace('.','') + "2017", format='%d%m%Y'))
USvideos['date'] = USvideos['date'].dt.date
USvideos = USvideos.drop_duplicates()
USvideos = USvideos.merge(categories, on='category_id')

In [9]:
ENGLISH_STOP_WORDS = frozenset([
    "a", "about", "above", "across", "after", "afterwards", "again", "against",
    "all", "almost", "alone", "along", "already", "also", "although", "always",
    "am", "among", "amongst", "amoungst", "amount", "an", "and", "another",
    "any", "anyhow", "anyone", "anything", "anyway", "anywhere", "are",
    "around", "as", "at", "back", "be", "became", "because", "become",
    "becomes", "becoming", "been", "before", "beforehand", "behind", "being",
    "below", "beside", "besides", "between", "beyond", "bill", "both",
    "bottom", "but", "by", "call", "can", "cannot", "cant", "co", "con",
    "could", "couldnt", "cry", "de", "describe", "detail", "do", "done",
    "down", "due", "during", "each", "eg", "eight", "either", "eleven", "else",
    "elsewhere", "empty", "enough", "etc", "even", "ever", "every", "everyone",
    "everything", "everywhere", "except", "few", "fifteen", "fifty", "fill",
    "find", "fire", "first", "five", "for", "former", "formerly", "forty",
    "found", "four", "from", "front", "full", "further", "get", "give", "go",
    "had", "has", "hasnt", "have", "he", "hence", "her", "here", "hereafter",
    "hereby", "herein", "hereupon", "hers", "herself", "him", "himself", "his",
    "how", "however", "hundred", "i", "ie", "if", "in", "inc", "indeed",
    "interest", "into", "is", "it", "its", "itself", "keep", "last", "latter",
    "latterly", "least", "less", "ltd", "made", "many", "may", "me",
    "meanwhile", "might", "mill", "mine", "more", "moreover", "most", "mostly",
    "move", "much", "must", "my", "myself", "name", "namely", "neither",
    "never", "nevertheless", "next", "nine", "no", "nobody", "none", "noone",
    "nor", "not", "nothing", "now", "nowhere", "of", "off", "often", "on",
    "once", "one", "only", "onto", "or", "other", "others", "otherwise", "our",
    "ours", "ourselves", "out", "over", "own", "part", "per", "perhaps",
    "please", "put", "rather", "re", "same", "see", "seem", "seemed",
    "seeming", "seems", "serious", "several", "she", "should", "show", "side",
    "since", "sincere", "six", "sixty", "so", "some", "somehow", "someone",
    "something", "sometime", "sometimes", "somewhere", "still", "such",
    "system", "take", "ten", "than", "that", "the", "their", "them",
    "themselves", "then", "thence", "there", "thereafter", "thereby",
    "therefore", "therein", "thereupon", "these", "they", "thick", "thin",
    "third", "this", "those", "though", "three", "through", "throughout",
    "thru", "thus", "to", "together", "too", "top", "toward", "towards",
    "twelve", "twenty", "two", "un", "under", "until", "up", "upon", "us",
    "very", "via", "was", "we", "well", "were", "what", "whatever", "when",
    "whence", "whenever", "where", "whereafter", "whereas", "whereby",
    "wherein", "whereupon", "wherever", "whether", "which", "while", "whither",
    "who", "whoever", "whole", "whom", "whose", "why", "will", "with",
    "within", "without", "would", "yet", "you", "your", "yours", "yourself",
    "yourselves"])

In [10]:
def gen_dataset(df, top_words=1000, pad_len=100):
    
    # Data Clean Up
    df.comment_text.replace({r"[^a-zA-Z' ]+": ''}, regex=True, inplace=True)
    df.comment_text = df.comment_text.str.lower()
    
    comments = df['comment_text'].tolist()
    
    # Remove exceding spaces and stopwords
    comments = [' '.join(str(r).split()) for r in comments]
    
    cleaned_comments = []
    for r in comments:
        aux = []
        for word in r.split():
            if not word in ENGLISH_STOP_WORDS:
                aux.append(word)
        cleaned_comments.append(' '.join(aux))
    
    comments = cleaned_comments
    
    # Word frequency
    word_freq = Counter() 
    for comment in comments:
        for word in comment.split():
            if not word in word_freq:
                word_freq[word] = 0
            word_freq[word] += 1
    
    # Map word to id
    word_to_id = {}
    if top_words:
        top = top_words
        most_common_words = word_freq.most_common(top_words)

        for i in range(top_words):
            word_to_id[most_common_words[i][0]] = top
            top -= 1
    else:
        top = len(word_freq)
        most_common_words = word_freq.most_common()

        for i in range(len(word_freq)):
            word_to_id[most_common_words[i][0]] = top
            top -= 1
    
    np.save("word_to_id.npy", word_to_id)
    
    # Convert comments
    max_len = 0
    
    X_data = []
    for comment in comments:
        # Get max comment length
        if len(comment.split()) > max_len:
            max_len = len(comment.split())
        
        aux = []
        for word in word_to_id:
            if not word in comment:
                aux.append(0)
            else:
                aux.append(1)
        
        X_data.append(aux)
    
    return X_data

### Vetor de caracteristicas

In [11]:
feature_vector = gen_dataset(UScomments, top_words=500)

### Gerando os clusters

In [56]:
from sklearn.cluster import KMeans
import numpy as np
kmeans = KMeans(n_clusters=7, random_state=0).fit(feature_vector)
UScomments = UScomments.join(pd.DataFrame({'cluster' : kmeans.labels_.tolist()}))

### Regularizando o cluster

In [57]:
CategoriesByVideo = dict(zip(USvideos.video_id, USvideos.category))
ClusterComments = UScomments.drop(columns=['comment_text', 'likes', 'replies'])
ClusterComments.video_id = ClusterComments.video_id.apply(lambda x: CategoriesByVideo[x])
ClusterComments = ClusterComments.rename(columns=dict(video_id='category'))
ClusterComments = ClusterComments.groupby('category').cluster.apply(lambda x: x.value_counts().head(1)).reset_index()
ClusterComments = ClusterComments.drop(columns=['cluster'])
ClusterComments = ClusterComments.rename(columns=dict(level_1='cluster'))
ClusterComments

,category,cluster
0,Autos & Vehicles,4
1,Comedy,2
2,Education,1
3,Entertainment,4
4,Film & Animation,5
5,Gaming,4
6,Howto & Style,4
7,Music,0
8,News & Politics,4
9,Nonprofits & Activism,1
